In [1]:
import pandas as pd
import numpy as np
import csv
import random
import datetime
import boto3
import schedule
import time
import import_ipynb
import nbimporter
from faker import Faker
from statistics import mean
from datetime import datetime, timedelta

In [2]:
faker = Faker()

def generate_conversation_data(start_date, end_date):
    data = []
    current_date = start_date
    while current_date <= end_date:
        data.append({
            'customer_id': faker.uuid4(),
            'conversation_id': faker.uuid4(),
            'agent_id': faker.uuid4(),
            'call_start': current_date,
            'call_end': current_date + timedelta(minutes=random.randint(1, 30)),
            'call_duration_sec': random.randint(30, 600),
            'call_status': random.choice(['Answered', 'Missed', 'Rejected', 'Hangup', 'Voicemail']),
            'transcript': faker.sentence(),
            'sentiment_score': random.uniform(0, 1),
            'keywords': [faker.word() for _ in range(random.randint(1, 5))],
            'created_at': datetime.now(),
            'updated_at': datetime.now()
        })
        current_date += timedelta(days=1)
    return data

# Set the start and end dates for generating data
start_date = datetime(2023, 5, 1)
end_date = datetime(2023, 6, 30)

# Generate conversation data
synthetic_data = generate_conversation_data(start_date, end_date)

# Create a DataFrame from the generated data
df = pd.DataFrame(synthetic_data)

# Group the data by date and save one file per day
for date, group in df.groupby(df['call_start'].dt.date):
    file_name = f"conversation_data_{date}.csv"
    group.to_csv(file_name, index=False)
    print(f"Data for {date} saved to {file_name}")


Data for 2023-05-01 saved to conversation_data_2023-05-01.csv
Data for 2023-05-02 saved to conversation_data_2023-05-02.csv
Data for 2023-05-03 saved to conversation_data_2023-05-03.csv
Data for 2023-05-04 saved to conversation_data_2023-05-04.csv
Data for 2023-05-05 saved to conversation_data_2023-05-05.csv
Data for 2023-05-06 saved to conversation_data_2023-05-06.csv
Data for 2023-05-07 saved to conversation_data_2023-05-07.csv
Data for 2023-05-08 saved to conversation_data_2023-05-08.csv
Data for 2023-05-09 saved to conversation_data_2023-05-09.csv
Data for 2023-05-10 saved to conversation_data_2023-05-10.csv
Data for 2023-05-11 saved to conversation_data_2023-05-11.csv
Data for 2023-05-12 saved to conversation_data_2023-05-12.csv
Data for 2023-05-13 saved to conversation_data_2023-05-13.csv
Data for 2023-05-14 saved to conversation_data_2023-05-14.csv
Data for 2023-05-15 saved to conversation_data_2023-05-15.csv
Data for 2023-05-16 saved to conversation_data_2023-05-16.csv
Data for

In [3]:
import pandas as pd
import numpy as np

# Read the conversation data from CSV files (assuming one file per day)
file_names = ['conversation_data_2023-05-01.csv', 'conversation_data_2023-06-02.csv']  # Add more file names as needed

data = pd.concat([pd.read_csv(file_name) for file_name in file_names], ignore_index=True)

# Calculate metrics for each agent
metrics = data.groupby('agent_id').agg({
    'call_duration_sec': ['mean', lambda x: np.percentile(x, 90)]
}).reset_index()

# Rename the columns for the metrics
metrics.columns = ['agent_id', 'average_call_length', '90th_percentile_call_length']

# Save the metrics to a CSV file
metrics.to_csv('metrics1.csv', index=False)

In [4]:
# Import the `upload_csv_to_s3` function from the notebook
from s3_upload import upload_csv_to_s3

# Specify the necessary parameters
csv_filename = 'metrics1.csv'
bucket_name = 'conversationcustomerdata'

# Call the upload_csv_to_s3 function
upload_csv_to_s3(csv_filename, bucket_name)

importing Jupyter notebook from s3_upload.ipynb


In [5]:
import datetime

def run_daily_task():
    synthetic_data = generate_conversation_data(start_date, end_date)
    print("Synthetic data generated.")
    
    metrics = calculate_metrics(synthetic_data)
    print("Metrics calculated.")
    
    csv_filename = 'metrics.csv'
    write_metrics_to_csv(metrics, csv_filename)
    print("Metrics written to CSV file.")
    
    bucket_name = 'conversationcustomerdata'
    upload_csv_to_s3(csv_filename, bucket_name)
    print("CSV file uploaded to S3.")

# Check if it's time to execute the task
current_time = datetime.datetime.now().strftime("%H:%M")
scheduled_time = "16:15"  # Set the desired scheduled time

if current_time >= scheduled_time:
    print("Executing daily task...")
    run_daily_task()
else:
    print("It's not yet time to execute the daily task.")

It's not yet time to execute the daily task.
